In [78]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from os import listdir
from os.path import isfile, join
import cv2
import pandas as pd
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image

In [73]:
folder_path=r'C:\Users\sures\Desktop\Python\face-emotion-recognition\images'
emotion_labels = ['angry','disgust','fear','happy','neutral', 'sad', 'surprise']
dict_emotion={'angry':0,'disgust':1,'fear':2,'happy':3,'neutral':4, 'sad':5, 'surprise':6}
print(dict_emotion['angry'])

0


In [72]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
emotion_classifier =load_model('model/model.h5')

In [55]:
def prepare_data(type_data):
    df= pd.DataFrame([])
    for emotion in emotion_labels:
        
        mypath=folder_path+'/'+type_data+'/'+emotion
        all_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
        #print(emotion,':',len(all_files))
        for img_file in all_files:
            if ('jpg' in img_file) | ('jpeg' in img_file)|('png' in img_file):
                data=[]
                data.append([mypath+'/'+img_file,emotion])
                
                df=df.append(data, ignore_index=True)
    return df

In [56]:
def detect_emotion(image_path):
    img = cv2.imread(image_path)
    labels = []
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        if True:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = emotion_classifier.predict(roi)[0]

            label=emotion_labels[prediction.argmax()]
            return label

In [74]:
X= pd.DataFrame([])
X=prepare_data("validation")

In [75]:
print(len(X))

10677


In [76]:
X.head(10)

,0,1
0,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry
1,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry
2,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry
3,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry
4,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry
5,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry
6,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry
7,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry
8,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry
9,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry


In [ ]:
y= pd.DataFrame([])
for index, row in X.iterrows():
    prediction=detect_emotion(row[0])
    #print(prediction)
    y=y.append([prediction], ignore_index=True)
    

In [69]:
y.head(1)

,0
0,fear


In [70]:
X['Prediction']=y

In [71]:
X.head(10)

,0,1,Prediction
0,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear
1,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear
2,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear
3,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear
4,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear
5,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear
6,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear
7,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear
8,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear
9,C:\Users\sures\Desktop\Python\face-emotion-rec...,angry,fear


In [63]:
X.to_csv('fer.csv')

In [52]:
for index, row in X.iterrows():
    if not row['Prediction']:
        os.remove(row[0])